In [1]:
from util.visualizer import Visualizer
from models import create_model
from data import create_dataset
from options.train.my_train_options import TrainOptions
import time
import sys
import torch
import torchvision
import os
from dotenv import load_dotenv

print(torch.__version__)
print(torchvision.__version__)

sys.argv = [""]
load_dotenv("config.env")

print(os.getenv("trainset_dir"))

2.0.1
0.15.2
E:/UIE_larry/UIE_larry/my_practice/datasets/mydatasets/my_aligned_train_dataset


In [2]:
import torch
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
opt = TrainOptions().parse()  # get training options

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: E:/UIE_larry/UIE_larry/my_practice/datasets/mydatasets/my_aligned_train_dataset
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest      

In [3]:
dataset = create_dataset(
    opt
)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)  # get the number of images in the dataset.
print("The number of training images = %d" % dataset_size)

model = create_model(opt)  # create a model given opt.model and other options
model.setup(opt)  # regular setup: load and print networks; create schedulers
# create a visualizer that display/save images and plots
visualizer = Visualizer(opt)
total_iters = 0  # the total number of training iterations

for epoch in range(
    opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1
):  # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()  # timer for data loading per iteration
    epoch_iter = (
        0  # the number of training iterations in current epoch, reset to 0 every epoch
    )
    # reset the visualizer: make sure it saves the results to HTML at least once every epoch
    visualizer.reset()
    # update learning rates in the beginning of every epoch.
    model.update_learning_rate()
    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt.print_freq == 0:
            t_data = iter_start_time - iter_data_time

        total_iters += opt.batch_size
        epoch_iter += opt.batch_size
        # unpack data from dataset and apply preprocessing
        model.set_input(data)
        # calculate loss functions, get gradients, update network weights
        model.optimize_parameters()

        if (
            total_iters % opt.display_freq == 0
        ):  # display images on visdom and save images to a HTML file
            save_result = total_iters % opt.update_html_freq == 0
            model.compute_visuals()
            visualizer.display_current_results(
                model.get_current_visuals(), epoch, save_result
            )

        if (
            total_iters % opt.print_freq == 0
        ):  # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if opt.display_id > 0:
                visualizer.plot_current_losses(
                    epoch, float(epoch_iter) / dataset_size, losses
                )

        if (
            total_iters % opt.save_latest_freq == 0
        ):  # cache our latest model every <save_latest_freq> iterations
            print(
                "saving the latest model (epoch %d, total_iters %d)"
                % (epoch, total_iters)
            )
            save_suffix = "iter_%d" % total_iters if opt.save_by_iter else "latest"
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if (
        epoch % opt.save_epoch_freq == 0
    ):  # cache our model every <save_epoch_freq> epochs
        print(
            "saving the model at the end of epoch %d, iters %d" % (epoch, total_iters)
        )
        model.save_networks("latest")
        model.save_networks(epoch)

    # print("Computing the following metrics for monitoring: PSNR and SSIM")
    # model_for_eval = model
    # model_for_eval.eval()
    # for i, data in enumerate(valid_dataset):
    #     if i >= opt.num_test:  # only apply our model to opt.num_test images.
    #         break
    #     model_for_eval.set_input(data)  # unpack data from data loader
    #     model_for_eval.foward()  # run inference

    print(
        "End of epoch %d / %d \t Time Taken: %d sec"
        % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time)
    )

dataset [AlignedDataset] was created
The number of training images = 3679
initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
---------- Networks initialized -------------
[Network G] Total number of parameters : 54.410 M
[Network D] Total number of parameters : 2.768 M
-----------------------------------------------
create web directory ./checkpoints\unet_rgb_L2\web...
learning rate 0.0002000 -> 0.0002000


e:\anaconda3\envs\inpainting-basic\Lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(epoch: 1, iters: 100, time: 0.083, data: 8.443) G_GAN: 0.470 G_L2: 0.271 D_real: 0.383 D_fake: 0.248 
(epoch: 1, iters: 200, time: 0.081, data: 0.001) G_GAN: 0.320 G_L2: 0.115 D_real: 0.328 D_fake: 0.197 
(epoch: 1, iters: 300, time: 0.081, data: 0.001) G_GAN: 0.351 G_L2: 0.055 D_real: 0.353 D_fake: 0.254 
(epoch: 1, iters: 400, time: 0.141, data: 0.002) G_GAN: 0.291 G_L2: 0.076 D_real: 0.294 D_fake: 0.274 
(epoch: 1, iters: 500, time: 0.083, data: 0.001) G_GAN: 0.262 G_L2: 0.066 D_real: 0.296 D_fake: 0.227 
(epoch: 1, iters: 600, time: 0.082, data: 0.001) G_GAN: 0.282 G_L2: 0.113 D_real: 0.273 D_fake: 0.256 


KeyboardInterrupt: 